In [ ]:
from random import random, choice, seed
import pickle

from merge_apps_util import merge
from instance import Instance, InfeasibleOnBuilding
from model import ModelSolver

# Generazione dataset di instanze di mapping di multiple applicazioni

In [ ]:
seed(42)

if False:
    path_apps, tree_apps, graph_apps = [], [], []
    for size in range(3,16):
        path_apps.append(f"path_{size:02d}_a")
        tree_apps.append(f"tree_{size:02d}_a")
        tree_apps.append(f"tree_{size:02d}_b")
        tree_apps.append(f"tree_{size:02d}_c")
        tree_apps.append(f"tree_{size:02d}_d")
        graph_apps.append(f"graph_{size:02d}_a")
        graph_apps.append(f"graph_{size:02d}_b")
        graph_apps.append(f"graph_{size:02d}_c")
        graph_apps.append(f"graph_{size:02d}_d")

    # for a in path_apps: print(a)
    # for a in tree_apps: print(a)
    # for a in graph_apps: print(a)

    ID = 0
    instances = []
    for n_size in range(30,91,10):
        for apps_count in range(2,31):

            for _ in range(3):

                n_suffix = choice(['a','b','c','d'])
                n_rp = choice([0,1,2,3,4])
                network_filename = f"DATASET_2/networks/network_{n_size:03d}_{n_suffix}.dat"
                network_rp_filename = f"DATASET_2/networks/network_{n_size:03d}_{n_suffix}_rp_{n_rp}.dat"

                app_filenames, app_rp_filenames = [], []
                for _ in range(apps_count): 
                    r = random()
                    if r < 0.2: app = choice(path_apps)     # 0.2 prob to choose path
                    elif r < 0.6: app = choice(tree_apps)   # 0.4 prob to choose tree
                    else: app = choice(graph_apps)          # 0.4 prob to choose graph
                    a_rp = choice((0,1,2,3,4))
                    app_filenames.append(f"DATASET_2/apps/{app}.dat")
                    app_rp_filenames.append(f"DATASET_2/apps/{app}_rp_{a_rp}.dat")

                output_app_filename = f"DATASET_2/apps_merged_FINAL2/id_{ID:04d}.dat"
                output_app_rp_filename = f"DATASET_2/apps_merged_FINAL2/id_{ID:04d}_rp.dat"
                ID += 1

                try:
                    merge(app_filenames, app_rp_filenames, output_app_filename, output_app_rp_filename)
                except Exception as e:
                    print("ERROR in merge, should not happen!")
                
                instances.append(
                    (network_filename, network_rp_filename, output_app_filename, output_app_rp_filename, app_filenames, app_rp_filenames)
                )

    # for i in instances:
    #     print(i)

    # with open('dumps/instances_dataset_FINAL.pickle', 'wb') as f:
    #    pickle.dump(instances, f, protocol=pickle.HIGHEST_PROTOCOL)

Le istanze sono salvate nel file :  instances_dataset_FINAL.pickle

In [ ]:
with open('dumps/instances_dataset_FINAL.pickle', 'rb') as f:
   instances = pickle.load(f)

#for i in instances: print(i)

# Root Node Relaxation

In [ ]:
if False:

    for idx,i in enumerate(instances):

        print()
        print(idx, len(i[4]), i)
        try:
            instance = Instance.build(
                network_filename=i[0], 
                network_rp_filename=i[1], 
                app_filename=i[2], 
                app_rp_filename=i[3]
            )
        except InfeasibleOnBuilding:
            print("INFEASIBLE on building")
            continue

        result_rnr = ModelSolver.optimize_model_at_root_node(instance)
        print(f"RNR {result_rnr}")
        
        if isinstance(result_rnr,tuple) and len(result_rnr) == 3:

            assert result_rnr[1] == "INFEASIBLE" or result_rnr[1] == "OPTIMAL"

# Compact Model con time limit 30 min delle istanze non RNR infeasible

In [ ]:
with open('dumps/rnr_feasible_instances_indexes.pickle', 'rb') as f:
   indexes = pickle.load(f)

In [ ]:
for network_size in indexes:
        
    for idx in indexes[network_size]:

        i = instances[idx]
        print()
        print(idx, len(i[4]), i)
        try:
            instance = Instance.build(
                network_filename=i[0], 
                network_rp_filename=i[1], 
                app_filename=i[2], 
                app_rp_filename=i[3]
            )
        except InfeasibleOnBuilding:
            print("INFEASIBLE on building")
            continue

        result_compact = ModelSolver.optimize_model(instance)
        print(f"COMPACT {result_compact}")